In [ ]:
We Need to Start this code at 10:30 in the morning The above analysis will stop execution @ 15:30

In [1]:
import numpy as np
from nsepython import *
import matplotlib.pyplot as plt
from IPython.display import clear_output
import pandas as pd
from win10toast import ToastNotifier
from nsetools import Nse
from datetime import datetime

In [ ]:
nse = Nse()
today_date=datetime.now().strftime("%d_%m")

In [ ]:
summary_df=pd.DataFrame(columns=['time','ltp','fut_ltp','Call','Put','Difference', 'change', 'pChange','call_vol','put_vol'])
list_of_columns_to_draw=summary_df.columns[1:]
len(list_of_columns_to_draw)

In [ ]:
def round_to_next100(n):
    return n + (100 - n) % 100

In [ ]:
def oi_analysis(summary_df):
    oi_data,ltp,crontime=oi_chain_builder('BANKNIFTY')
#     global summary_df
    price=round_to_next100(ltp)
    fut_ltp=nse_quote_ltp("BANKNIFTY","latest","Fut")

    ul=price+700
    ll=price-700
    subset_oi=oi_data.loc[(oi_data['Strike Price']<=ul) & (oi_data['Strike Price']>=ll)]
    app_dict={'time' : crontime[12:17],"ltp":ltp,'fut_ltp':fut_ltp,'Call' : np.sum(subset_oi['CALLS_Chng in OI']), 'Put' : np.sum(subset_oi['PUTS_Chng in OI']),
                             'Difference' : np.sum(subset_oi['CALLS_Chng in OI'])-np.sum(subset_oi['PUTS_Chng in OI']),'call_vol':np.sum(subset_oi['CALLS_Volume']),'put_vol':np.sum(subset_oi['PUTS_Volume'])}
    nb_quote=nse.get_index_quote("nifty bank")
    for i in ['change','pChange']:
        app_dict.update({i:nb_quote[i]})
#     print(crontime[:11])
    return(summary_df.append(app_dict,ignore_index = True),subset_oi)

In [ ]:
# # ,x,y,x_name
def plot_subplot(axiss,column_name):
#     print(axis[0])
    axis[axiss[0], axiss[1]].plot(summary_df['time'],summary_df[column_name])
    axis[axiss[0], axiss[1]].tick_params(axis='x', rotation=90)
    axis[axiss[0], axiss[1]].grid() 
    axis[axiss[0], axiss[1]].set_title(column_name)

In [ ]:
toaster = ToastNotifier()

while summary_df.iloc[-1]['time'].split(":")[0] != '15:30':
    summary_df,subset_oi=oi_analysis(summary_df)
#     summary_df=pd.read_csv('C:/Users/shiwa/Algo trading/Data/Saved Data/Complete_BANKNIFTY_DATA/data_BANKNIFTY_08_07_1419.csv')
    clear_output(wait=False)
#     toaster.show_toast("Python Notification","take a screen shot")
#     summary_df.to_csv('C:/Users/shiwa/Algo trading/Data/Saved Data/Complete_BANKNIFTY_DATA/data_BANKNIFTY_'+today_date+"_"+str(summary_df['time'][len(summary_df['time'])-1]).replace(':','')+'.csv')      
#     subset_oi.to_csv('C:/Users/shiwa/Algo trading/Data/Saved Data/Complete_BANKNIFTY_DATA/OI_data_BANKNIFTY_'+today_date+"_"+str(summary_df['time'][len(summary_df['time'])-1]).replace(':','')+'.csv')      
    print(summary_df)
    plt.rcParams["figure.figsize"] = (20,15)
    figure, axis = plt.subplots(int(np.ceil(len(list_of_columns_to_draw)/3)),3)
    figure.tight_layout(pad=5.0)
    for i,j in zip([(i,j) for j in range(3) for i in range(2)],list_of_columns_to_draw):
        plot_subplot(i,j)
#     plt.savefig('C:/Users/shiwa/Algo trading/Data/Saved Data/Charts/analysis_chart_'+today_date+"_"+str(summary_df['time'][len(summary_df['time'])-1]).replace(':','')+'.png')
    plt.show()
    time.sleep(600)